# Spectrogen

# To Do

### Old


1. ~~Make list~~
2. ~~[Get GPU working?](https://pytorch.org/docs/stable/notes/faq.html)~~
3. ~~Change model for generator Transposed convolution~~
4. Compress using discriminator
5. Debug training to find where memory goes WHOOOOOP


4. Fix Checkerboard Pattern
5. Train on single Genre
6. Upgrade our Discriminator (More CNN)
7. "Tunning" 

    a. Stride

    b. ~~CNN Nonsense~~
8. Run that shit all day
9. Train to generate specific Genere
10. Increase image resolution 


Resolution Improvement
1. Skip connections
2. Learning rate
3. New Datase
4. More Conv Layers between upsacles
4. Slow down upscaling
5. Go bigger then scale down
6. Max pool?  
5. ?


# Preperation

## Imports

In [ ]:
import fastbook
from fastbook import *

import re
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import gc

from PIL import Image

import librosa
import librosa.display
import IPython.display as ipd

import soundfile as sf

import random

from datetime import datetime

import numpy as np

## Variables

In [ ]:
device = "cuda"

model_selection = "ProGan"
#model_selection = "SmoothGan"

batch_number = 10 # 4

epoch_number = 250

label_smoothing = False

disc_lr=0.0001
gen_lr=0.0001

z_dim = 10

og_img_size =  [batch_number, z_dim , 4, 4] #32
dimensions = (batch_number, 1, 512, 512)
noise_vec = [batch_number, z_dim]

init_alpha =.5
inc_alpha = .1

print_every = 1

print_layers = False


progan_seq = [4, 8, 16, 32, 64, 128, 256, 512]

prodisc_seq = progan_seq[::-1]
max_num_blocks = len(progan_seq)

## Setup

In [ ]:
#path = Path.BASE_PATH = r"./squash_mel"
path = Path.BASE_PATH = r"./train_half"

music = DataBlock(blocks = (ImageBlock(cls=PILImageBW)),
        get_items= get_image_files,
        splitter = RandomSplitter(seed = 23))

dls = music.dataloaders(path, bs=batch_number)

music.summary(path)

# Discriminator

In [ ]:
class Discriminator_conv(nn.Module):
    def __init__(self):
        super(Discriminator_conv, self).__init__()
        self.discriminator = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels =16, kernel_size=3, stride=2),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels =16, out_channels =32,  kernel_size=3, stride=2),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels =32, out_channels =64,  kernel_size=3, stride=2),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels =64, out_channels =32,  kernel_size=3, stride=2),
            
            nn.Conv2d(in_channels =32, out_channels =1,  kernel_size=3, stride=2),
        )
        
    def forward(self, image):
        b = self.discriminator(image).view(batch_number, -1)
        return b

#discriminator = Discriminator_conv().cuda()
#batch_x = dls.valid.one_batch()[0]
#x = batch_x.cuda()
#out = discriminator(batch_x)
#print(out)
#out = out.view(batch_number, -1)
#out.shape

# Generator

### Utilities

In [ ]:
class Reshape(nn.Module):
    """
    Class for performing a reshape as a layer in a sequential model.
    """
    def __init__(self, shape=[]):
        super(Reshape, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(x.size(0), *self.shape)
    
    def extra_repr(self):
            # (Optional)Set the extra information about this module. You can test
            # it by printing an object of this class.
            return 'shape={}'.format(
                self.shape
            )

In [ ]:
def view_gen_image(y):
    y = y.clone()
    image = y[0,0].detach().to("cpu").numpy() 
    plt.figure(figsize=(10,10))
    plt.imshow(image, cmap="gray")

    
class PrintLayer(nn.Module):
    def __init__(self, label):
        super(PrintLayer, self).__init__()
        self.label = label

    def forward(self, x):
        if print_layers:
            print(self.label)
            print(x.shape)
            view_gen_image(x)
        return x

In [ ]:
def make_image(gen):
    x = torch.rand(noise_vec).to("cuda")
    y = gen(x)
    
    image = torch.reshape(y, dimensions)
    
    y = y.detach().to("cpu").numpy()
    y.ndim
    
    image = image[0,0].detach().to("cpu").numpy() 
    
    plt.figure(figsize=(8,8))
    plt.imshow(image, cmap = "gray")
    plt.show()

### Current AreTwo

In [ ]:
class AreTwoGenFourSmoothSquare(nn.Module):
    def __init__(self):
        super().__init__()
        self.generator = nn.Sequential(
            nn.Linear(noise_vec[1], np.prod(og_img_size[1:])),
            nn.BatchNorm1d(np.prod(og_img_size[1:])),
            nn.ReLU(True),
        
            nn.Linear(np.prod(og_img_size[1:]), np.prod(og_img_size[1:])),
            nn.BatchNorm1d(np.prod(og_img_size[1:])),
            nn.ReLU(True),
            
            Reshape(og_img_size[1:]),
            
            PrintLayer("Pre Conv2"),

            nn.Conv2d(in_channels = og_img_size[1],  out_channels = 1024, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(num_features=1024, momentum = 0.7),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2),
            
            PrintLayer("Chunk1"),

            nn.Conv2d(in_channels = 1024,  out_channels = 512, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(num_features=512, momentum = 0.7),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2),
            
            PrintLayer("Chunk2"),

            nn.Conv2d(in_channels = 512,  out_channels = 256, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(num_features=256, momentum = 0.7),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2),
            
            PrintLayer("Chunk3"),

            nn.Conv2d(in_channels = 256,  out_channels = 128, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(num_features=128, momentum = 0.7),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2),
            
            PrintLayer("Chunk4"),

            nn.Conv2d(in_channels = 128,  out_channels = 64, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(num_features=64, momentum = 0.7),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2),
            
            
            PrintLayer("Chunk5"),
            
            nn.Conv2d(in_channels = 64,  out_channels = 32 , kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(num_features=32, momentum = 0.7),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2),
            
            PrintLayer("Chunk6"),
            
            nn.Conv2d(in_channels = 32,  out_channels = 16 , kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(num_features=16, momentum = 0.7),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2),
            
            PrintLayer("Chunk6"),
            
            nn.Conv2d(in_channels = 16,  out_channels = 1, kernel_size=3, stride=1, padding=0, bias=False),

            PrintLayer("Chunk6"),

        )

    def forward(self, x):
        x = self.generator(x)
        return x

def test_gen():
    x = torch.rand(noise_vec).cuda()
    print(x.shape)
    gen = AreTwoGenFourSmoothSquare().cuda()
    y = gen(x)
    print(y.shape)
    view_gen_image(y)
    return y

#y = test_gen()    
#y = y.clone()
#image = y[0,0].detach().to("cpu").numpy() 
#print(image.shape)

## ProGan

In [ ]:
class ProConGan(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.ModuleList([ProGanBlock(z_dim, 512)])
        self.alpha = init_alpha
        self.alpha_increment = inc_alpha
        self.toBW =  nn.ModuleList([nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, stride=1, bias=False).cuda()])
        self.dim_progression = progan_seq
        self.model_size = len(self.dim_progression)-1
        self.dim_index = 0

    def add_block(self):
        self.model.append(nn.Upsample(scale_factor=2))
        in_shape = self.dim_progression[self.model_size-(self.dim_index)]
        out_shape =self.dim_progression[max(0,self.model_size-(self.dim_index+1))]
        
        self.model.append(ProGanBlock(in_channels=in_shape, \
            out_channels=out_shape).cuda())

        if len(self.model) == 3:
            self.toBW.append(nn.Conv2d(in_channels=out_shape, out_channels=1, kernel_size=1, stride=1, bias=False).cuda())
        else:
            self.toBW[0] = self.toBW[1]
            self.toBW[1] = nn.Conv2d(in_channels=out_shape, out_channels=1, kernel_size=1, stride=1, bias=False).cuda()
        

        self.reset_alpha()
        self.dim_index += 1
        #Replace to BW

    def increment_alpha(self):
        self.alpha = self.alpha+self.alpha_increment
        self.alpha = min(1, self.alpha)
    
    def reset_alpha(self):
        self.alpha = init_alpha

    def get_dim(self):
        return self.dim_progression[self.dim_index]

    def forward(self, x):
        for i, layer in enumerate(self.model):
            if i<len(self.model)-1 or len(self.model) == 1:
                x = layer(x)
                if len(self.model) == 1:
                    #print(x.shape)
                    x = self.toBW[0](x)
            else:
                x = self.toBW[1](self.alpha*layer(x)) + self.toBW[0]((1-self.alpha)*(x))
        return x

class ProGanBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(),
        )
    def forward(self, x):
        x = self.block(x)
        return x
        

def test_progan():
    pro = ProConGan().cuda()
    x = torch.rand([10,1,4,4]).cuda()
    for i in range(8):
        y = pro(x)
        print(y.shape)
        pro.add_block()
        pro.increment_alpha()
    view_gen_image(y)
    print(pro)

In [ ]:
class ProConDisc(nn.Module):
    def __init__(self):
        super().__init__()
        #self.model = nn.ModuleList([ProDiscBlock(1, 4), nn.Flatten(), nn.Linear(int(np.prod(og_img_size)/batch_number), 1)])
        self.model = nn.ModuleList([ProDiscBlock(2, 5), 
                                    FinalDiscBlock(2, 1),
                                    FinalDiscBlock(1, 1)
                                   ])
        self.alpha = init_alpha
        self.alpha_increment = inc_alpha
        self.fromBW =  nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=4, kernel_size=1, stride=1).cuda()])
        self.dim_progression = prodisc_seq
        self.dim_index=0
        self.model_size = len(self.dim_progression)-1 

    def add_block(self):
        in_shape = self.dim_progression[self.model_size-(self.dim_index)]
        out_shape =self.dim_progression[max(0,self.model_size-(self.dim_index+1))]

        if len(self.model) == 3:
            self.fromBW.append(nn.Conv2d(in_channels=1, out_channels=out_shape, kernel_size=1, stride=1).cuda())
        else:
            self.fromBW[0] = self.fromBW[1]
            self.fromBW[1] = nn.Conv2d(in_channels=1, out_channels=out_shape, kernel_size=1, stride=1).cuda()

        self.model.insert(0,nn.AvgPool2d(kernel_size=2, stride=2))
        self.model.insert(0,ProDiscBlock(in_shape, out_shape).cuda())
        self.reset_alpha()
        self.dim_index+=1

        

    def minibatch_std(self, x):
        batch_statistics = (
            torch.std(x, dim=0).mean().repeat(x.shape[0], 1, x.shape[2], x.shape[3])
        )
        # we take the std for each example (across all channels, and pixels) then we repeat it
        # for a single channel and concatenate it with the image. In this way the discriminator
        # will get information about the variation in the batch/image
        return torch.cat([x, batch_statistics], dim=1)
        
    def increment_alpha(self):
        self.alpha = self.alpha+self.alpha_increment
        self.alpha = min(1, self.alpha)
    
    def reset_alpha(self):
        self.alpha = init_alpha

    def forward(self, x):
        #model[0] is the CNN layer
        #model[1] is the avg pool layer
        if len(self.model) > 3:
    
            x_1 = self.fromBW[1](x)
            x_1 = self.model[0](x_1)
            x_1 = self.model[1](x_1)

            x_2 = self.model[1](x)
            x_2 = self.fromBW[0](x_2)
            x = self.alpha * x_1 + (1-self.alpha)*x_2
            
        for i, layer in enumerate(self.model):
            if len(self.model) == 3 and i ==0:
                x = self.fromBW[0](x)
                x = self.minibatch_std(x)
            if i>1 or len(self.model) == 3: 
                if i+3 == len(self.model) and len(self.model) !=3:
                    x = self.minibatch_std(x)
                x = layer(x) 
        return x[:,:,0,0]
    
    
class ProDiscBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            PixelNorm(),
            nn.Conv2d(in_channels=out_channels, out_channels=in_channels, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            PixelNorm(),
        )
    def forward(self, x):
        x = self.block(x)
        return x
        
class FinalDiscBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
        )
    def forward(self, x):
        x = self.block(x)
        return x

class PixelNorm(nn.Module):
    def __init__(self):
        super(PixelNorm, self).__init__()
        self.epsilon = 1e-8

    def forward(self, x):
        return x / torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + self.epsilon)
    


def test_discgan():
    pro = ProConGan().cuda()
    disc = ProConDisc().cuda()
    x = torch.rand([10,1,4,4]).cuda()
    batch_x = dls.valid.one_batch()[0]
    batch_x =downsample(7, batch_x)
    #print(disc(batch_x))
    
    for i in range(3):
        print("EPOCH")
        y = pro(x)
        result = disc(y)
        pro.add_block()
        disc.add_block()
        pro.increment_alpha()
    
#test_discgan()

In [ ]:
class DownDownDown(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.AvgPool2d(2,2)
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
##assert(False)

# Losses

### Utils

In [ ]:
def positive_label_smoothing(labels):
    if label_smoothing:
        return labels - torch.rand(labels.shape)*.1
    else:
        return labels
    
def negative_label_smoothing(labels):
    if label_smoothing:
        return labels + torch.rand(labels.shape)*.1
    else:
        return labels

In [ ]:
def downsample(n_times, image):
    pool =  DownDownDown()
    for n in range(n_times):
        image = pool(image)
    return image
       

### Discriminator Loss

In [ ]:
def get_noise(num_images, z_dim, device="cuda"):
    noise = None
    if device == "cuda":
        if model_selection != "ProGan":
            noise = torch.rand(noise_vec , device=device).cuda()
        else:
            noise = torch.rand(og_img_size , device=device).cuda()
    else:
        noise = torch.rand(noise_vec)
    return noise

In [ ]:
def get_disc_loss(gen, disc, criterion, real_input, num_images, z_dim, device="cuda"):
    '''
    Return Loss of discriminator 
    Params: 
    Gen: Geneartor model
    Disc: Discrimiator Model
    criterion: Loss function
    Real Input: Our data set images
    Num_images: Len(real input)
    z_dim: Dimention of our noise vector
    Return a loss value for 1 batch
    '''

    real_input = downsample((7-disc.dim_index), real_input)
    
    fake_images = gen(get_noise(num_images, z_dim, device=device))
    if model_selection == "ProGan":
        fake_images = torch.reshape(fake_images, (batch_number, 1, gen.get_dim(), gen.get_dim()))
    else: 
        fake_images = torch.reshape(fake_images, dimensions)
    fake_images.detach()
    
    fake_labels = negative_label_smoothing(torch.zeros((num_images,1),device=device))
    real_labels = positive_label_smoothing(torch.ones((num_images,1),device=device))                                    

    fake_loss = criterion(disc(fake_images), fake_labels)
    real_loss = criterion(disc(real_input), real_labels)
    disc_loss = (fake_loss + real_loss)/2
    return disc_loss

In [ ]:
def get_disc_loss_wgan(gen, disc, real_images, device="cuda"):
    fake_images = gen(get_noise(num_images, z_dim, device=device))
    fake_images = torch.reshape(fake_images, dimensions)
    fake_images.detach()

    fake_pred = disc(fake_images)
    real_pred = disc(real_images)
    disc_loss = -(torch.mean(real_pred) - torch.mean(fake_pred))
    return disc_loss


### Generator Loss

In [ ]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device="cuda"):
    '''
    Returns the loss of the generator
    Params: 
    Gen: Generator Model
    Disc: Discrimiator Model
    Criterion: Loss Function
    num_images: The number of images the generator should produce
    z_dim: The dim of the noise vector
    Device: What device we run it on
    '''
    fake_images = gen(get_noise(num_images, z_dim, device=device))
    if model_selection == "ProGan":
        fake_images = torch.reshape(fake_images, (batch_number, 1, gen.get_dim(), gen.get_dim()))
    else: 
        fake_images = torch.reshape(fake_images, dimensions)
    fake_images.detach()
    
    real_labels = positive_label_smoothing(torch.ones((num_images,1),device=device)) 
    
    prediction = disc(fake_images)
    gen_loss = criterion(prediction, real_labels)
    return gen_loss

In [ ]:
def get_gen_loss_wgan(gen, disc, real_images, device="cuda"):
    fake_images = gen(get_noise(num_images, z_dim, device=device))
    fake_images = torch.reshape(fake_images, dimensions)
    fake_images.detach()

    fake_pred = disc(fake_images)
    gen_loss = -torch.mean(fake_pred)

    return gen_loss

# Training

### Loss functions to test

In [ ]:
#criterion = nn.BCELoss
# what we've been using

#criterion = nn.BCEWithLogitsLoss()
# whub whub converges at 40

#criterion = nn.CrossEntropyLoss()
# doesn't work

#criterion = nn.L1Loss()
# chirpy and musical really quiet without the whub whub converges at 20

#criterion = nn.CTCLoss()
# TypeError: forward() missing 2 required positional arguments: 'input_lengths' and 'target_lengths'

#criterion = nn.NLLLoss()
# RuntimeError: 0D or 1D target tensor expected, multi-target not supported

#criterion = nn.PoissonNLLLoss()
# so bad, basically nothing

#criterion = nn.GaussianNLLLoss()
# TypeError: forward() missing 1 required positional argument: 'var'

#criterion = nn.KLDivLoss()
# UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.
# negative numbers lol
# demonic shit

#criterion = nn.MarginRankingLoss()
# vTypeError: forward() missing 1 required positional argument: 'target'

#criterion = nn.HingeEmbeddingLoss()
# waste of time

#criterion = nn.MultiLabelMarginLoss
# RuntimeError: Boolean value of Tensor with more than one value is ambiguous

#criterion = nn.HuberLoss()
# literally nothing

#criterion = nn.SmoothL1Loss()
# literally nothing

#criterion = nn.SoftMarginLoss()
# converges on the first epoch

#criterion = nn.MultiLabelSoftMarginLoss()
# okay 90s kids tv show computer

#criterion = nn.CosineEmbeddingLoss()
# TypeError: forward() missing 1 required positional argument: 'target'

#criterion = nn.MultiMarginLoss()
# RuntimeError: inconsistent target size, expected 4 but got [4, 1]

#criterion = nn.TripletMarginLoss()
# TypeError: forward() missing 1 required positional argument: 'negative'

# criterion = nn.TripletMarginWithDistanceLoss()
# TypeError: forward() missing 1 required positional argument: 'negative'

In [ ]:
now = datetime.now()
current_time = now.strftime("%y%d%H%M%S")

out_path = os.path.join("./", current_time)
os.mkdir(out_path)

def make_image4(gen, name):
    x = get_noise(num_images, z_dim, device=device).to("cuda")
    y = gen(x)
    y = y[0,0].detach().to("cpu").numpy()
    
    plt.figure(figsize=(8,8))
    plt.imshow(y, cmap = "gray")
    plt.savefig(out_path+"/Generated"+str(name)+".png")
    plt.show()
    np.save(out_path+"/Generated"+str(name), y)

    return y

In [ ]:
def disc_metric(disc, gen):
    reals = []
    fakes = []
    for i in range(25):
        reals.append(np.mean(disc(dls.valid.one_batch())))
        fakes.append(np.mean(disc(gen(get_noise(num_images, z_dim, device=device)))))
    return np.mean(reals), np.mean(fakes)

In [ ]:
assert(True) #lol

## Current training

In [ ]:
print_layers = False
criterion = nn.MSELoss()

# converges at 14 amazing whub but melodic A+

gan = None
disc = None

block_count = 0

if model_selection == "ProGan":
    gen = ProConGan().to(device)
    disc = ProConDisc().to(device)
elif model_selection == "SmoothGan":
    gen = AreTwoGenFourSmoothSquare().to(device)
    disc = Discriminator_conv().to(device)

#NON WGAN
gen_opt = torch.optim.Adam(gen.parameters(), lr=gen_lr)
disc_opt = torch.optim.Adam(disc.parameters(), lr=disc_lr)

#WGAN
#gen_opt = torch.optim.RMSprop(gen.parameters())
#disc_opt = torch.optim.RMSprop(disc.parameters())


disc_loss_hist = [] 
gen_loss_hist = []

disc_metric_real = []
disc_metric_fake = []


for epoch in range(epoch_number):
    disc_loss_batch = 0.0
    gen_loss_batch = 0.0
    for batch in dls.train:
        real_images, *_ = batch
        real_images = real_images.to(device)
        num_images = len(real_images)

        #Train Generator
        gen_opt.zero_grad()
        
        gen_loss = get_gen_loss(gen, disc, criterion, num_images, z_dim, device)
        #gen_loss = get_gen_loss_wgan(gen, disc, real_images)
        
        gen_loss_batch+=gen_loss.item()
        gen_loss.backward(retain_graph=True)
        gen_opt.step()

        ##Train discriminator
        disc_opt.zero_grad()
        
        disc_loss = get_disc_loss(gen, disc, criterion, real_images, num_images, z_dim, device)
        #disc_loss = get_disc_loss_wgan(gen, disc, real_images)
        
        disc_loss_batch+=disc_loss.item()
        disc_loss.backward(retain_graph=True)
        disc_opt.step()
        
        if model_selection == "ProGan":
            gen.increment_alpha()
            disc.increment_alpha()


    print("Finished epoch")
    print("Alpha: "+ str(gen.alpha))
    if epoch % 10==0 and epoch !=0 and gen.dim_index !=8 and block_count < max_num_blocks:
        gen.add_block()
        disc.add_block()
        print("adding block")
        block_count += 1


    print("Epoch: " + str(epoch))
    print(disc_loss_batch/batch_number)
    print((gen_loss_batch/batch_number)) #WHY?

    disc_loss_hist.append(disc_loss_batch/batch_number)
    gen_loss_hist.append(gen_loss_batch/batch_number)

    if epoch % (print_every) == 0: 
        for i in range(1):
            make_image4(gen,epoch)
            reals = downsample((7-disc.dim_index), real_images)
            reals = reals.detach().to("cpu").numpy()[0][0]
            print(reals.shape)
            fake_images = gen(get_noise(num_images, z_dim, device=device))
            #fake_images = torch.reshape(fake_images, dimensions)
            #fake_images.detach()
            print("REAL")
            plt.figure(figsize=(8,8))
            plt.imshow(reals, cmap = "gray")
            plt.show()
            
            fake_images = torch.reshape(fake_images, (batch_number, 1, gen.get_dim(), gen.get_dim()))
            fake_images.detach()
            prediction = disc(fake_images)
            print(prediction)
   #if epoch % 10: 
   #    real, fake = disc_metric(disc, gen)
   #    disc_metric_real.append(real)
   #    disc_metric_fake.append(fake)

### After training

In [ ]:
fig,ax = plt.subplots()
ax.plot(disc_loss_hist)

ax.set_xlim(0,200)
ax.set_ylim(1.9, 2.1)

ax.plot(np.array(gen_loss_hist))
fig.show()

In [ ]:
print_layers = True
x = torch.rand(noise_vec).to("cuda")
y = gen(x)

In [ ]:
def load_numpy(filename):
    gen_out = np.load(filename)
    image = gen_out[0,0]
    
    return image

def make_image3(gen):
    x = torch.rand(noise_vec).to("cuda")
    y = gen(x)

    image = torch.reshape(y, dimensions)


    image = image[0,0].detach().to("cpu").numpy()

    plt.figure(figsize=(10,10))
    plt.imshow(image, cmap = "gray")

    im = Image.fromarray(image).convert('RGB')
    im.save("Generated.jpeg")

    return image

def create_wav_file(image, file_name):
    S = librosa.feature.inverse.mel_to_stft(image, norm='slaney')
    y = librosa.griffinlim(S)
    
    sf.write(file_name, y, 22050)

##image = make_image3(gen)
print_layers = [286, 274, 242, 190, 183, 156, 137, 128, 111, 101, 91, 62, 27]

for layer in print_layers:
    gen_out = load_numpy(out_path+"/Generated"+str(layer)+".npy")
    create_wav_file(gen_out,str(layer)+".wav")

In [ ]:
gen_out.shape

In [ ]:
create_wav_file(image, "test.wav")

In [ ]:
y, sr = librosa.load(r"C:\Users\ldscho\spectro\ml_expore\test.wav", duration=23)
S = np.abs(librosa.stft(y))

S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
plt.figure()
librosa.display.specshow(S_db)
plt.colorbar()

# Librosa

In [ ]:
def audio_to_spectrogram_and_back():
    x, sr = librosa.load(r"C:\Users\ldscho\Downloads\2.wav", duration=11.87)

    S = librosa.feature.melspectrogram(n_mels=512, y=x, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', power=2.0, fmin=75, fmax=4000)
    x_inv = librosa.feature.inverse.mel_to_audio(M=S, sr=22050, n_fft=2048, hop_length=512, n_iter=32, win_length=None, window='hann', center=True, pad_mode='constant', power=2.0, fmin=75, fmax=4000)

    S = S.astype(np.uint8)
    im = Image.fromarray(S)
    im.save("S.png")

    sf.write("first.wav", x_inv, sr)

    return x_inv, S

a= audio_to_spectrogram_and_back()

In [ ]:
def mel_generate(path):
    #[^\]*(?=[.][a-zA-Z]+$)
    pattern = re.compile(r'(.+)(\\)(?P<name>.+)(.wav)')
    for music_name in os.listdir(path):
        file_name = os.path.join(path, music_name)
        if os.path.isfile(file_name):
            match = list((re.finditer(pattern, file_name)))
            name = match[0].groupdict()['name']
            print(file_name)

            #a = random.uniform(0,12)
            #b = random.uniform(0,12)
            #c = random.uniform(0,12)
            #d = random.uniform(0,12)

            a = 0
            b = 3
            c = 6
            d = 9

            w, sr = librosa.load(file_name, duration=11.87, offset=a)
            x, sr = librosa.load(file_name, duration=11.87, offset=b)
            y, sr = librosa.load(file_name, duration=11.87, offset=c)
            z, sr = librosa.load(file_name, duration=11.87, offset=d)

            W = librosa.feature.melspectrogram(n_mels=512, y=w, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', power=2.0, fmin=75, fmax=4000)
            X = librosa.feature.melspectrogram(n_mels=512, y=x, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', power=2.0, fmin=75, fmax=4000)
            Y = librosa.feature.melspectrogram(n_mels=512, y=y, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', power=2.0, fmin=75, fmax=4000)
            Z = librosa.feature.melspectrogram(n_mels=512, y=z, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', power=2.0, fmin=75, fmax=4000)

            lM_W = W.astype(np.uint8)
            im_W = Image.fromarray(lM_W)
            lM_X = X.astype(np.uint8)
            im_X = Image.fromarray(lM_X)
            lM_Y = Y.astype(np.uint8)
            im_Y = Image.fromarray(lM_Y)
            lM_Z = Z.astype(np.uint8)
            im_Z = Image.fromarray(lM_Z)


            im_W.save("a_" + name + ".png")
            im_X.save("b_" + name + ".png")
            im_Y.save("c_" + name + ".png")
            im_Z.save("d_" + name + ".png")


mel_generate(r"C:\Users\ldscho\spectro\ml_expore\converted")

# Too far lmao

In [ ]:
#   .....                                                                                           
# ^PYJJJYYJ?!^.                                                                                     
# P5     .:~!?YYJ?^.                                                                                
# 5G           .:!JY57.                 :!^                                                         
# :#!               ^YB!             ~J55JGP.                                                       
#  7#.                ^G5:          :&?.   5B                                                       
#   BJ                  ?5Y!.       J#     .PP^                                                     
#   JB                    ^JG7      PY       !G5.                                                   
#    PP                     :YG^    5P~J:  .J! B5                                                   
#    .PP                      ~PY.  ^@P!.   :: ~@^                                                  
#     .BJ            :~:        ?B!  J#^        BY             .::^~~:                              
#       .BJ           ^YG5!.      ^BJ  J#:       ?B        .!Y5YYYJJ7?GY:                            
#       :PY~.          .~JPGY!^:. .JG^^@~       7#     ~?5PY!.       .5#^                           
#        ~JP?^         :?GY!?JY55J?PBGJ        GP ..?PP7:             J&.                          
#           .!55^      ~&?:      .::...        .P5JJY5^            :^!!Y#:                          
#              ~GY.    !&                                   .^!JJJY5?7!!.                           
#                ?G7  .B5                                 ~YGJ7~:..                                 
#                 .5G.:@:                               ^PP!.                                       
#                   PP:&^                             .:BJ                                          
#                   .PGB5                             !G@7.                                         
#                     ?B@GY?J7!!!!!~                   J#JG?.                                       
#                      .?BY^^!!!!!!G5.                 GY :JP5~                                     
#                        :YG?      .?#~                PY    ~G5^                                   
#                          :YG~      #7                PY      !G?.                                 
#                            ~P5^   ^&:                PY       .?5Y~                               
#                              ?#7  ?B                 PY          ~PJ:                             
#                               ^YP7BJ                 PY     .^~77!7P&G7^                          
#                   .~~!?7?!~:.   .P@P:                GY .~?P5J?7~~~~75GBGJ^                       
#                 :55J7!~^!7JYY5?~ ^@7.                J&5PY~:            ?@#Y.                     
#                 GP           :7Y5YG:                 .?!.                #7?G7                    
#                 #7                                                      7#^ :5P~                  
#                 #7                                                     J#^    ~P5!                
#                 5B:                                             .::^~?P5:      ~?YPJ:             
#                 .Y5YYY?7!^^^:........:^^^^.         ..:!????JJJYY5JJ?!^        ~5P7?P7            
#                     .^^!7??JYJJJJJJJJYJ??JB~       ?BJYJ!^^^^...                 ~PY.             
#                                           BJ       !&                              JB^            
#                                           G5       ?@7.                             ?#.           
#                                           ~&^      :&P5Y^                           .&^           
#                                            &~       PP !GY.                         :&^           
#                                           .&!       ~&.  YG!                        .&^           
#                                            PP       .&?   ^5P7.                     JB.           
#                                            ~&^       ?#     ^?PJ7:                .JB^            
#                                             BJ       !&.      .^?YYJ~^.         :?PJ.             
#                                             JB       Y#           .~?JYYJ?7~~~?557:               
#                                             7#       !&.               .:~!777!:                  
#                                             7#       7&                                           
#                                             ^@~     .#J                                           
#                                              GP     5G                                            
#                                              :BY.  ?&:                                            
#                                               .5GYPP^                                             
#                                                 :7^   